<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/adv_cv/class_2/10%20-%20%20Laborat%C3%B3rio/C3_W2_Lab_1_Simple_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Visão computacional avançada com TensorFlow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow?specialization=tensorflow-advanced-techniques) de [Laurence Moroney](https://laurencemoroney.com/) e [Andrew Ng](https://www.coursera.org/instructor/andrewng) , [DeepLearning.AI](https://www.deeplearning.ai/)

# Detecção simples de objetos no Tensorflow

Neste laboratório, você aprenderá a usar os modelos de detecção de objetos disponíveis no [Tensorflow Hub](https://www.tensorflow.org/hub).

Nas seções a seguir, você vai:
* explorar o Tensorflow Hub para modelos de detecção de objetos
* carregar os modelos em seu espaço de trabalho
* pré-processar uma imagem para inferência 
* executar a inferência nos modelos e inspecionar a saída

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

### Baixar o modelo do Tensorflow Hub

O Tensorflow Hub é um repositório de modelos de aprendizado de máquina treinados que você pode reutilizar em seus próprios projetos. 
- Você pode ver os domínios cobertos [aqui](https://tfhub.dev/) e suas subcategorias. 
- Para este laboratório, você deverá examinar a subcategoria[`image-object-detection`](https://tfhub.dev/s?module-type=image-object-detection). 
- Você pode selecionar um modelo para ver mais informações sobre ele e copiar o URL para fazer o download em seu espaço de trabalho. 
- Utilizaremos um [inception resnet version 2](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1)
- Você também pode modificar a célula a seguir para escolher o outro modelo: [ssd mobilenet version 2](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2)

In [ ]:
# você pode trocar as linhas comentadas aqui para escolher o outro modelo

# Início da versão 2 do resnet
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# Em vez disso, você pode escolher a versão 2 do ssd mobilenet e comparar os resultados
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

#### Carregar o modelo

Em seguida, você carregará o modelo especificado pelo `module_handle`.
- Isso levará alguns minutos para carregar o modelo.

In [ ]:
model = hub.load(module_handle)

#### Escolha a assinatura padrão

Alguns modelos no hub do Tensorflow podem ser usados para diferentes tarefas.

Portanto, a documentação de cada modelo deve mostrar qual *assinatura* usar ao executar o modelo. 
- Se quiser ver se um modelo tem mais de uma assinatura, você pode fazer algo como `print(hub.load(module_handle).signatures.keys())`.

Aqui os modelos que você usará têm apenas a assinatura `default`, portanto, você não precisa se preocupar com outros tipos.

In [ ]:
# Dê uma olhada nas assinaturas disponíveis para esse modelo específico
model.signatures.keys()

Escolha a assinatura "padrão" para seu detector de objetos.

- Para modelos de detecção de objetos, sua assinatura "padrão" aceitará um lote de tensores de imagem e produzirá um dicionário que descreve os objetos detectados, que é o que você deseja aqui.

In [ ]:
detector = model.signatures['default']

### download_and_resize_image

Essa função faz o download de uma imagem especificada por uma determinada "url", pré-processa-a e, em seguida, salva-a no disco.

In [ ]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Obtém uma imagem on-line, redimensiona-a e salva-a localmente.
    
    Args:
        url (string) -- link para a imagem
        new_width (int) -- tamanho em pixels usado para redimensionar a largura da imagem
        new_height (int) -- tamanho em pixels usado para redimensionar o comprimento da imagem
        
    Retorna:
        (string) -- caminho para a imagem salva
    '''
    
    
    # cria um arquivo temporário que termina com ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # abre o URL fornecido
    response = urlopen(url)
    
    # lê a imagem obtida do URL
    image_data = response.read()
    
    # Coloca os dados da imagem no buffer de memória
    image_data = BytesIO(image_data)
    
    # abre a imagem
    pil_image = Image.open(image_data)

    # Redimensiona a imagem. Será cortada se a proporção for diferente.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.Resampling.LANCZOS)
    
    # converte para o espaço de cores RGB
    pil_image_rgb = pil_image.convert("RGB")
    
   # salva a imagem no arquivo temporário criado anteriormente
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Imagem baixada para %s." % filename)
    
    return filename

### Baixar e pré-processar uma imagem

Agora, usando `download_and_resize_image`, você pode obter uma imagem de amostra on-line e salvá-la localmente. 
- Forneci uma URL para você, mas fique à vontade para escolher outra imagem para passar pelo detector de objetos.
- Você pode usar a largura e a altura originais da imagem, mas fique à vontade para modificá-la e ver os resultados obtidos.

In [ ]:
# Você pode escolher um URL diferente que aponte para uma imagem de sua escolha
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# Faça o download da imagem e use a altura e a largura originais
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

### run_detector

Essa função receberá o modelo de detecção de objetos `detector` e o caminho para uma imagem de amostra e, em seguida, usará esse modelo para detectar objetos e exibir suas categorias de classe previstas e caixas de detecção.
- run_detector usa `load_image` para converter a imagem em um tensor.

In [ ]:
def load_img(path):

    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img


def run_detector(detector, path):
    '''
    Carrega uma imagem JPEG e a converte em um tensor.
    
    Args:
        path (string) -- caminho para uma imagem JPEG salva localmente
    
    Retorna:
        (tensor) -- um tensor de imagem
    '''
    
    # lê o arquivo
    img = load_img(path)

    # Adicionar uma dimensão de lote na frente do tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # Executar a inferência usando o modelo
    result = detector(converted_img)

   # Salvar os resultados em um dicionário
    result = {key:value.numpy() for key,value in result.items()}

    # imprimir resultados
    print("Encontrados %d objetos." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


### Executar inferência na imagem

Você pode executar seu detector chamando a função `run_detector`. Isso imprimirá o número de objetos encontrados seguido de três listas: 

* As pontuações de detecção de cada objeto encontrado (ou seja, o grau de confiança do modelo), 
* As classes de cada objeto encontrado, 
* As caixas delimitadoras de cada objeto

Você verá como sobrepor essas informações à imagem original nas próximas seções e na atividade avaliativa!

In [ ]:
# Executa o modelo de detecção de objetos e imprime informações sobre os objetos encontrados
run_detector(detector, downloaded_image_path)